In [1]:
import torch
import torch.nn as nn
from myutils.common import yaml_util
from utils import module_util

In [2]:
config_file_path = '../config/densenet169.yaml'
config = yaml_util.load_yaml_file(config_file_path)
model = module_util.get_model(config, 'cpu')

In [3]:
modules = list()
module_util.extract_all_child_modules(model, modules, extract_designed_module=False)

In [4]:
for i, module in enumerate(modules):
    print(i, type(module).__name__)

0 Conv2d
1 BatchNorm2d
2 ReLU
3 MaxPool2d
4 _DenseBlock
5 _Transition
6 _DenseBlock
7 _Transition
8 _DenseBlock
9 _Transition
10 _DenseBlock
11 BatchNorm2d
12 ReLU
13 AvgPool2d
14 Linear


In [9]:
x = torch.ones(1, 3, 224, 224)
start_idx = 0
end_idx = 8
teacher_model = nn.Sequential(*modules[start_idx:end_idx])
z = teacher_model(x)
z.size()

torch.Size([1, 256, 14, 14])

In [6]:
student_model = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.Conv2d(64, 128, kernel_size=2, stride=1, bias=False),
        nn.AvgPool2d(kernel_size=2, stride=2, padding=1)
)
z = student_model(x)
z.size()

torch.Size([1, 128, 28, 28])